In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance
import pydeck as pdk
import pickle
import requests
import json

In [2]:
model = pickle.load(open('../../../capstone_other/files_too_large_for_github/gradient_boosting_1.pickle', 'rb'))

In [3]:
data = pd.read_csv('../../../capstone_other/files_too_large_for_github/ML_table_with_availability_clusters.csv')

In [4]:
new = data.groupby('dock_id').apply(pd.DataFrame.sample, random_state = 5, n=1).reset_index(drop=True)

In [5]:
new.to_csv('../dash_app/data/robert/new.csv')

In [6]:
new1 = new[['dock_id', 'latitude', 'longitude', 'tot_docks']]

In [7]:
new2 = new[['weekday_cluster', 'weekend_cluster']]

In [8]:
request = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat=40.7812&lon=-73.9665&exclude=minutely,current,daily,alerts&units=metric&appid=404310456b8e1c31228341dd6c95dd04')
response = request.json()

In [9]:
weather_df = pd.DataFrame.from_dict(response['hourly'])[['dt', 'temp','pop']]
weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit = 's')

In [10]:
weather_df['pop'] = weather_df['pop'].apply(lambda x: round(x))

In [11]:
day_of_week_conversion = {0:2, 1:3, 2:4, 3:5, 4:6, 5:7, 6:1}

In [12]:
full = pd.DataFrame()
for row in list(weather_df.iterrows()):
    to_append = pd.DataFrame()
    date = row[1]['dt']
    month = date.month
    to_append['month'] = np.ones(len(new), dtype=int) * month
    day_of_week = date.dayofweek
    sql_day_of_week = day_of_week_conversion[day_of_week]
    to_append['num_day'] = np.ones(len(new), dtype=int) * sql_day_of_week
    hour = date.hour
    to_append['hour'] = np.ones(len(new), dtype=int) * hour
    to_append = pd.concat([to_append, new1], axis = 1)
    temp = row[1]['temp']
    rain = row[1]['pop']
    to_append['tmp'] = np.ones(len(new)) * temp
    to_append['rain'] = np.ones(len(new), dtype=int) * rain
    to_append['datetime'] = [date for _ in range(len(new))]
    to_append = pd.concat([to_append, new2], axis = 1)
    full = pd.concat([full, to_append], axis = 0)

In [13]:
dock_id_series = full['dock_id']
datetime_series = full['datetime']

In [14]:
full.drop(['dock_id', 'datetime'], axis = 1, inplace = True)

In [15]:
predictions = model.predict(full)

In [16]:
full['predictions'] = predictions
full['dock_id'] = dock_id_series
full['datetime'] = datetime_series

In [17]:
full['avail_bikes_proportion'] = full['predictions']/full['tot_docks']

In [18]:
user_input = pd.to_datetime('2021-12-19 14:00:00')

In [19]:
month = user_input.month
num_day = day_of_week_conversion[user_input.dayofweek]
hour = user_input.hour

In [20]:
full[(full['month'] == month) & (full['num_day'] == num_day) & (full['hour'] == hour)]

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,datetime,avail_bikes_proportion


In [21]:
full.to_csv('../dash_app/data/robert/dataframe_for_live_predictions.csv', index = False)

In [22]:
full

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,datetime,avail_bikes_proportion
0,12,2,17,40.683826,-73.976323,62,1.17,0,2,3,30.998591,83,2021-12-20 17:00:00,0.499977
1,12,2,17,40.696089,-73.978034,19,1.17,0,2,0,12.537537,119,2021-12-20 17:00:00,0.659870
2,12,2,17,40.686768,-73.959282,19,1.17,0,1,0,3.501500,120,2021-12-20 17:00:00,0.184289
3,12,2,17,40.692395,-73.993379,24,1.17,0,2,3,20.409130,143,2021-12-20 17:00:00,0.850380
4,12,2,17,40.698399,-73.980689,19,1.17,0,2,0,12.484769,144,2021-12-20 17:00:00,0.657093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,12,4,16,40.805726,-73.936322,33,6.19,0,2,2,9.868827,3505,2021-12-22 16:00:00,0.299055
325,12,4,16,40.801307,-73.939817,29,6.19,0,0,2,2.357537,3506,2021-12-22 16:00:00,0.081294
326,12,4,16,40.804555,-73.939686,36,6.19,0,0,2,16.745264,3507,2021-12-22 16:00:00,0.465146
327,12,4,16,40.801194,-73.950074,31,6.19,0,2,0,19.332126,3509,2021-12-22 16:00:00,0.623617


# Load .csv file for live predictions

In [43]:
full = pd.read_csv('../dash_app/data/robert/dataframe_for_live_predictions.csv')

In [44]:
month = 12
num_day = 2
hour = 23

In [45]:
query = full[(full['month'] == month) & (full['num_day'] == num_day) & (full['hour'] == hour)]

In [46]:
def manhattan_distance(start_lat, start_lon, end_lat, end_lon):
    dist = distance((start_lat, start_lon), (start_lat, end_lon)).miles + \
           distance((end_lat, end_lon), (start_lat, end_lon)).miles
    return dist

In [47]:
low_bike_threshold = 1/3
high_bike_threshold = 2/3
max_distance = 2
max_bikes_rebalanced = 500
min_cargo_size = 3

In [48]:
data_low = query[query['avail_bikes_proportion'] <= low_bike_threshold]

In [49]:
data_high = query[query['avail_bikes_proportion'] >= high_bike_threshold]

In [50]:
data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')

<ipython-input-50-d746bb208678>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


In [51]:
data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')

<ipython-input-51-d9c66eeb9a9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')


In [52]:
data_low = data_low.sort_values(by = 'deficit', ascending = False)

In [53]:
data_high = data_high.sort_values(by = 'surplus', ascending = False)

In [54]:
rebalancing_dict = {}

In [55]:
low_copy = data_low.copy()
high_copy = data_high.copy()

In [56]:
bikes_rebalanced = 0
for low in low_copy.index:
    if low_copy.loc[low, 'deficit'] == 0:
        continue
    for high in high_copy.index:
        if high_copy.loc[high, 'surplus'] == 0:
            continue
        if manhattan_distance(low_copy.loc[low, 'latitude'], low_copy.loc[low, 'longitude'], 
                              high_copy.loc[high, 'latitude'], high_copy.loc[high, 'longitude']) < max_distance:
            stations_key = (low_copy.loc[low, 'dock_id'], high_copy.loc[high, 'dock_id'])
            change = min(low_copy.loc[low, 'deficit'], high_copy.loc[high, 'surplus'])
            low_copy.loc[low, 'deficit'] -= change
            high_copy.loc[high, 'surplus'] -= change
            bikes_rebalanced += change
            if stations_key in rebalancing_dict.keys():
                rebalancing_dict[stations_key] += change
            else:
                rebalancing_dict[stations_key] = change
            if low_copy.loc[low, 'deficit'] == 0:
                break

In [57]:
rebalancing_dict

{(3141, 3296): 10,
 (3141, 3318): 6,
 (3141, 3136): 3,
 (3134, 3136): 2,
 (3134, 3165): 3,
 (3134, 3168): 3,
 (3134, 3124): 2,
 (3134, 3236): 2,
 (3457, 448): 2,
 (469, 3259): 4,
 (524, 462): 13,
 (195, 3311): 7,
 (195, 331): 5,
 (3139, 3312): 5,
 (3362, 3387): 5,
 (3362, 3379): 3,
 (3362, 3425): 2,
 (167, 462): 2,
 (167, 297): 6,
 (167, 334): 2,
 (3452, 3109): 9,
 (3452, 3096): 1,
 (3462, 334): 2,
 (3153, 3338): 3,
 (3301, 3496): 6,
 (3301, 3500): 2,
 (3304, 3419): 8,
 (319, 331): 2,
 (319, 264): 5,
 (319, 408): 1,
 (3075, 3096): 5,
 (3075, 291): 3,
 (3345, 3500): 3,
 (3345, 3494): 5,
 (3146, 3425): 1,
 (3146, 3390): 2,
 (3146, 3374): 2,
 (3146, 3391): 1,
 (3284, 3502): 4,
 (3284, 3495): 3,
 (2012, 3224): 5,
 (2012, 369): 2,
 (3299, 3497): 2,
 (3126, 3123): 5,
 (491, 369): 3,
 (491, 368): 3,
 (3469, 389): 4,
 (3469, 439): 1,
 (143, 3324): 5,
 (436, 3364): 4,
 (3456, 3072): 2,
 (3456, 3057): 1,
 (3179, 3047): 1,
 (337, 3388): 3,
 (3423, 3324): 3,
 (364, 3410): 3,
 (249, 408): 3,
 (3232

In [58]:
sorted_rebalancing = dict(sorted(rebalancing_dict.items(), key=lambda x: x[1], reverse = True))

In [59]:
filtered_rebalancing = {key: value for key, value in sorted_rebalancing.items() if value >= min_cargo_size}

In [60]:
filtered_rebalancing

{(524, 462): 13,
 (3141, 3296): 10,
 (3452, 3109): 9,
 (3304, 3419): 8,
 (195, 3311): 7,
 (3141, 3318): 6,
 (167, 297): 6,
 (3301, 3496): 6,
 (195, 331): 5,
 (3139, 3312): 5,
 (3362, 3387): 5,
 (319, 264): 5,
 (3075, 3096): 5,
 (3345, 3494): 5,
 (2012, 3224): 5,
 (3126, 3123): 5,
 (143, 3324): 5,
 (469, 3259): 4,
 (3284, 3502): 4,
 (3469, 389): 4,
 (436, 3364): 4,
 (3141, 3136): 3,
 (3134, 3165): 3,
 (3134, 3168): 3,
 (3362, 3379): 3,
 (3153, 3338): 3,
 (3075, 291): 3,
 (3345, 3500): 3,
 (3284, 3495): 3,
 (491, 369): 3,
 (491, 368): 3,
 (337, 3388): 3,
 (3423, 3324): 3,
 (364, 3410): 3,
 (249, 408): 3}

In [61]:
filtered_bikes_rebalanced = 0
final_rebalancing_dict = {}
for k, v in filtered_rebalancing.items():
    if filtered_bikes_rebalanced < max_bikes_rebalanced:
        final_rebalancing_dict[k] = v
        filtered_bikes_rebalanced += v

In [62]:
rebalancing_df = pd.DataFrame(final_rebalancing_dict.items(), columns = ['dock_ids', 'num_bikes'])

In [63]:
rebalancing_df

,dock_ids,num_bikes
0,"(524, 462)",13
1,"(3141, 3296)",10
2,"(3452, 3109)",9
3,"(3304, 3419)",8
4,"(195, 3311)",7
5,"(3141, 3318)",6
6,"(167, 297)",6
7,"(3301, 3496)",6
8,"(195, 331)",5
9,"(3139, 3312)",5


In [64]:
rebalancing_df[['dock_id_receive', 'dock_id_give']] = rebalancing_df['dock_ids'].tolist()
rebalancing_df.drop(['dock_ids'], axis = 1, inplace = True)

In [65]:
rebalancing_df

,num_bikes,dock_id_receive,dock_id_give
0,13,524,462
1,10,3141,3296
2,9,3452,3109
3,8,3304,3419
4,7,195,3311
5,6,3141,3318
6,6,167,297
7,6,3301,3496
8,5,195,331
9,5,3139,3312


In [50]:
data_df = query[['dock_id', 'latitude', 'longitude']]

In [51]:
new_docks = new[['dock_id', 'dock_name']]

In [52]:
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_receive', 'longitude': 'longitude_receive'})
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_give', 'longitude': 'longitude_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [53]:
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_receive'})
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [54]:
GREEN_RGB = [0, 255, 0, 150]
RED_RGB = [240, 100, 0, 150]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data = rebalancing_df,
    get_width="num_bikes",
    get_source_position=["longitude_give", "latitude_give"],
    get_target_position=["longitude_receive", "latitude_receive"],
    get_tilt=15,
    get_source_color=GREEN_RGB,
    get_target_color=RED_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=40.74, longitude=-74, bearing=290, pitch=50, zoom=12)


TOOLTIP_TEXT = {"html": "{num_bikes} bikes need rebalancing from<br />{dock_name_give} to {dock_name_receive}"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT, map_style = 'light')
r.to_html("monday_12pm_rebalancing.html")
with open('monday_12pm_rebalancing.json', 'w') as outfile:
    json.dump(r.to_json(), outfile)

In [55]:
bikes_rebalanced

215

In [56]:
filtered_bikes_rebalanced

168

In [57]:
data_low['deficit'].sum()

554

In [58]:
data_high['surplus'].sum()

401

In [59]:
data_low.head(5)

,Unnamed: 0,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,datetime,avail_bikes_proportion,deficit
8369,144,12,2,23,40.765005,-73.958185,59,3.35,0,5,1,0.193985,3141,2021-12-20 23:00:00,0.003288,19
8363,138,12,2,23,40.763126,-73.965269,51,3.35,0,0,1,1.601765,3134,2021-12-20 23:00:00,0.031407,15
8522,297,12,2,23,40.763026,-73.972095,46,3.35,0,5,1,-0.081139,3457,2021-12-20 23:00:00,-0.001764,15
8304,79,12,2,23,40.763441,-73.982681,57,3.35,0,0,1,5.061485,469,2021-12-20 23:00:00,0.088798,14
8523,298,12,2,23,40.763094,-73.978350,40,3.35,0,5,1,0.379920,3458,2021-12-20 23:00:00,0.009498,13


In [60]:
data_high.head(5)

,Unnamed: 0,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,datetime,avail_bikes_proportion,surplus
8303,78,12,2,23,40.746920,-74.004519,47,3.35,0,4,0,45.895549,462,2021-12-20 23:00:00,0.976501,15
8408,183,12,2,23,40.782454,-73.948920,42,3.35,0,2,0,37.948021,3296,2021-12-20 23:00:00,0.903524,10
8348,123,12,2,23,40.726060,-73.956210,31,3.35,0,1,0,29.413954,3109,2021-12-20 23:00:00,0.948837,9
8531,306,12,2,23,40.655400,-74.010628,33,3.35,0,2,0,29.562206,3477,2021-12-20 23:00:00,0.895824,8
8503,278,12,2,23,40.679279,-73.981540,27,3.35,0,3,3,25.897488,3419,2021-12-20 23:00:00,0.959166,8


In [61]:
table = rebalancing_df[['dock_name_give', 'dock_name_receive', 'num_bikes']]
table = table.rename(columns = {'dock_name_give': 'dock origin', 'dock_name_receive': 'dock destination', 'num_bikes': 'number of bikes'})

table_html = table.to_html()

text_file = open("rebalancing_table.html", "w")
text_file.write(table_html)
text_file.close()

In [62]:
rebalancing_df.head()

,num_bikes,dock_id_receive,dock_id_give,latitude_receive,longitude_receive,latitude_give,longitude_give,dock_name_receive,dock_name_give
0,13,524,462,40.755273,-73.983169,40.746920,-74.004519,W 43 St & 6 Ave,W 22 St & 10 Ave
1,10,3141,3296,40.765005,-73.958185,40.782454,-73.948920,1 Ave & E 68 St,E 93 St & 2 Ave
2,9,3452,3109,40.719156,-73.948854,40.726060,-73.956210,Bayard St & Leonard St,Banker St & Meserole Ave
3,8,3304,3419,40.668127,-73.983776,40.679279,-73.981540,6 Ave & 9 St,Douglass St & 4 Ave
4,7,195,3311,40.709056,-74.010434,40.687632,-74.001626,Liberty St & Broadway,Columbia St & Kane St
